In [ ]:
from load_data import * 
from model import * 
from testing_utils import * 
from train_utils import *

## Load Data

In [ ]:

loaders = create_pets_loaders(
    root="./data",
    img_size=(256,256),
    batch_size=12,
    num_workers=0,
    binary=True,
    augment=True,
    val_split=0.15)

for imgs, masks in loaders["train"]:
    print(imgs.shape, masks.shape)
    break


## Create Model

In [ ]:
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet(in_channels=3, num_classes=1).to(device)
opt = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()


## Train Model

In [ ]:
epochs = 30

for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    train_metrics = train_epoch_seg(loaders['train'], model, opt, criterion, num_classes=1, device=device, amp=True)
    val_metrics = evaluate_seg(loaders['val'], model, criterion, num_classes=1, device=device)

## Test Model

In [ ]:
clear_all_hooks(model)

In [ ]:
model.eval()
with torch.no_grad():
    xb, yb = next(iter(loaders['val']))
    xb1, yb1 = xb[:1].to(device), yb[:1].to(device)
    out = model(xb1)

imagenet_mean = (0.485, 0.456, 0.406)
imagenet_std  = (0.229, 0.224, 0.225)

viz_overlay_errors(xb1, out, yb1, thr=0.5, mean=imagenet_mean, std=imagenet_std)

In [ ]:
model.eval()
with torch.no_grad():
    xb, yb = next(iter(loaders['val']))
    xb, yb = xb.to(device), yb.to(device)
    logits = model(xb)
plot_pr_roc_from_logits(logits, yb)

In [ ]:
model.eval()
all_dice, all_iou = [], []
with torch.no_grad():
    for xb, yb in loaders['val']:
        xb, yb = xb.to(device), yb.to(device)
        logits = model(xb)
        all_dice.extend(dice_per_image_from_logits(logits, yb))
        all_iou.extend(iou_per_image_from_logits(logits, yb))
plot_hist_metrics(np.array(all_dice), np.array(all_iou))

In [ ]:
with torch.no_grad():
    xb, yb = next(iter(loaders['val']))
    xb, yb = xb.to(device), yb.to(device)
    logits = model(xb)
calibration_curve_pixels(logits, yb)

In [ ]:
model.eval()
with torch.no_grad():
    xb, yb = next(iter(loaders['val']))
    xb = xb[:2].to(device)

# Si normalizaste con ImageNet (cámbialo si usaste otros stats)
imagenet_mean = (0.485, 0.456, 0.406)
imagenet_std  = (0.229, 0.224, 0.225)

mean, var = mc_dropout_predict(model, xb, T=20, num_classes=1)
plot_uncertainty(xb.cpu(), mean.cpu(), var.cpu(), idx=0, mean_std=(imagenet_mean, imagenet_std))

In [ ]:
xb, _ = next(iter(loaders['val']))
xb1 = xb[:1].to(device)
# ejemplo: una capa concreta de tu modelo
visualize_feature_maps(model, layer=model.enc[0], xb=xb1, num_maps=12)

In [ ]:
hook_layer = model.bottleneck if hasattr(model, "bottleneck") else list(model.modules())[-2]
X, Y = collect_bottleneck(model, loaders['val'], device, hook_layer)
plot_embedding_2d(X, Y=None, method="umap")

In [ ]:
with torch.no_grad():
    xb, yb = next(iter(loaders['val']))
    xb1, yb1 = xb[:1].to(device), yb[:1].to(device)
    logits = model(xb1)
prob = torch.sigmoid(logits)[0,0].cpu().numpy()
pred = (prob>0.5).astype(np.uint8)

gt = yb1[0]
if gt.dim()==3: gt = gt[0]
gt = gt.cpu().numpy().astype(np.uint8)

bf1, bprec, brec = boundary_f1(pred, gt, tol=2)
hd = hausdorff_distance(pred, gt)
print("Boundary-F1:", bf1, "Hausdorff:", hd)

In [ ]:
with torch.no_grad():
    xb, yb = next(iter(loaders['val']))
heat = occlusion_sensitivity(model, xb[:1], yb[:1], patch=24, stride=12, device=device)

## Pablo 
